In [5]:
workset_id = '664d5a4c520000e4020a706c'
import http.client
import json
import ssl
from collections import defaultdict
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict
import pandas as pd
from jinja2 import Template
from IPython.display import HTML



# Connect to the HTRC API
conn = http.client.HTTPSConnection("tools.htrc.illinois.edu", context = ssl._create_unverified_context())

# Set the headers for the API request
headers = { 'Content-Type': "application/json" }

# Make an API request to get information about the volumes in the workset
conn.request("GET", "/ef-api/worksets/664d5a4c520000e4020a706c/metadata")

# Get the response and parse the data
res = conn.getresponse()
data = res.read()
data = json.loads(data.decode("utf-8"))
#data



/Library/Frameworks/Python.framework/Versions/3.10/bin/python3.10


In [6]:
df = pd.DataFrame({
    "Year": [item["metadata"]["pubDate"] for item in data["data"]],
    "Publication Title": [item["metadata"]["title"] for item in data["data"]],
    "Publisher Name": [
        ", ".join(pub["name"] for pub in item["metadata"].get("publisher", [])) if isinstance(item["metadata"].get("publisher", {}), list)
        else item["metadata"].get("publisher", {}).get("name", "Publisher Unknown")
        for item in data["data"]
    ],
    "Publication Place": [item["metadata"].get("pubPlace", {}).get("name", "Place Unknown") for item in data["data"]],
})

grouped_full_data = df.groupby('Year').apply(lambda x: x.sort_values(by='Publication Place', ascending=False)).reset_index(drop=True)
full_data_for_template = {str(year): group.to_dict(orient='records') for year, group in grouped_full_data.groupby('Year')}
top_publications_df = df.groupby('Year').apply(lambda x: x.head(4)).reset_index(drop=True)
top_data_for_template = {str(year): group.to_dict(orient='records') for year, group in top_publications_df.groupby('Year')}
total_publications_per_year = {str(year): len(records) for year, records in full_data_for_template.items()}

html_template = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Yearly Publications</title>
    <style>
        table, th, td { border: 1px solid black; border-collapse: collapse; }
        th, td { padding: 10px; }
        th { background-color: #f2f2f2; }
        .hidden { display: none; }
    </style>
    <script>
        function toggleVisibility(year) {
            var elements = document.querySelectorAll('.extra-publication-' + year);
            for (var i = 0; i < elements.length; i++) {
                elements[i].style.display = 'table-row';
            }
            document.getElementById('toggle-button-' + year).style.display = 'none';
        }
    </script>
</head>
<body>
    {% for year, records in top_data_for_template.items() %}
    <h3>{{ year }} ({{ total_counts[year] }} Total Publications)
        {% if total_counts[year] > 4 %}
            <button id="toggle-button-{{ year }}" onclick="toggleVisibility('{{ year }}')">Show All</button>
        {% endif %}
    </h3>
    <table>
        <tr>
            <th>Publication Title</th>
            <th>Publisher Name</th>
            <th>Publication Place</th>
        </tr>
        {% for row in records %}
        <tr>
            <td>{{ row['Publication Title'] }}</td>
            <td>{{ row['Publisher Name'] }}</td>
            <td>{{ row['Publication Place'] }}</td>
        </tr>
        {% endfor %}
        {% for row in full_data_for_template[year] %}
        {% if loop.index > 4 %}
        <tr class="extra-publication-{{ year }} hidden">
            <td>{{ row['Publication Title'] }}</td>
            <td>{{ row['Publisher Name'] }}</td>
            <td>{{ row['Publication Place'] }}</td>
        </tr>
        {% endif %}
        {% endfor %}
    </table>
    <hr>
    {% endfor %}
</body>
</html>
"""

template = Template(html_template)
rendered_html = template.render(top_data_for_template=top_data_for_template, full_data_for_template=full_data_for_template, total_counts=total_publications_per_year)

# Save and display the HTML
with open('visualization.html', 'w') as f:
    f.write(rendered_html)
display(HTML(rendered_html))

/var/folders/y8/p2b8sjnd36z34ktm8ll312g40000gn/T/ipykernel_56098/3271919415.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_full_data = df.groupby('Year').apply(lambda x: x.sort_values(by='Publication Place', ascending=False)).reset_index(drop=True)
/var/folders/y8/p2b8sjnd36z34ktm8ll312g40000gn/T/ipykernel_56098/3271919415.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top_publications_d

Publication Title,Publisher Name,Publication Place
"Pindarou Olympia, Pythia, Nemea, Isthmia ... Pindari Olympia, Pythia, Nemea, Isthmia.",Opera & impensa Petri Brubacchij,Germany
Publication Title,Publisher Name,Publication Place
"Ioannis Bocatij de Certaldo ... De casibus virorum illustrium libri nouem : hic liber iam olim etiam, sed antiquissimis, incultisque characteribus impressum, et nunc primum ab innumeris ... mendis /",Philippus Vlhardus excudebat,Germany
Publication Title,Publisher Name,Publication Place
"Petri Victorii Oratio habita ad Iulium III, initio pontificatus ipsius ...",apud Laurentium Torrentinum,Italy
Publication Title,Publisher Name,Publication Place
M. Terentii Varronis pars librorum quattuor et viginti de lingua latina ...,"apud Vincentium Luchinum, apud Antonium Bladum impressorem Cameralem",Italy
Publication Title,Publisher Name,Publication Place
De gli hecatommithi /,Appresso Girolamo Scotto.,Italy
Publication Title,Publisher Name,Publication Place
